In [1]:
from collections import defaultdict, namedtuple
import re
from typing import Dict, List

with open("input.txt") as f:
    lines = [line.strip() for line in f.readlines()]

In [2]:
NumberOfBags = namedtuple(
    "NumberOfBags", ["num", "color"]
)
# map a color to a list of bags that it contains
# i.e. this line
# "dark orange bags contain 2 shiny gold bags,
# 9 faded blue bags."
# becomes this in the dictionary
# color_contains["dark orange"] = [
#   NumberOfBags(2, "shiny gold"),
#   NumberOfBags(9, "faded blue")
# ]
color_contains: Dict[str, List] = defaultdict(list)

In [3]:
# Use regular expressions to parse each bag rule.
# The rule:
# "dark orange bags contain 2 shiny gold bags,
# 9 faded blue bags."
# will be matched by the first regular expression
# which we will use to just extract "dark orange"
# The second regular expression will extract the
# inner bags number and colors with re.findall
# which will return this list from the matching groups
# [("2", "shiny gold"), ("9", "faded blue")]
# These will be put into NumberOfBags objects
# to construct the dictionary example above
color_regex = "(\w+ \w+) bags contain"
num_and_color_regex = "(\d+) (\w+ \w+) bags?[,.]"
for line in lines:
    match = re.match(color_regex, line)
    outer_color = match.group(1)
    for bag in re.findall(num_and_color_regex, line):
        num, inner_color = int(bag[0]), bag[1]
        color_contains[outer_color].append(
            NumberOfBags(num, inner_color)
        )

## part 1

The question is how many bags can eventually contain
at least one shiny gold bag. We can model this as a
graph problem where each node points to the bag colors
that it can sit in. Then, we solve the question by
finding all nodes (i.e. bag colors) that can be reached
starting at "shiny gold".

First, we construct the graph as a dictionary where
each color maps to the other colors that it can reside
in, kind of the reverse of the `color_contains`
dictionary.

In [4]:
color_to_outer = defaultdict(list)

for outer_color, inner_colors in color_contains.items():
    for color in inner_colors:
        color_to_outer[color.color].append(outer_color)

In [5]:
visited = set()
to_visit = ["shiny gold"]

while to_visit:
    inner_color = to_visit.pop()
    if inner_color in visited:
        continue

    visited.add(inner_color)

    for outer_color in color_to_outer[inner_color]:
        if outer_color not in visited:
            to_visit.append(outer_color)

# subtract 1 because "shiny gold" is in visited,
# but we want to put it in at least one other
# bag
len(visited) - 1

229

## part 2

Part 2 asks how many bags must be in the shiny gold
bag. We can do this recursively by looking at how
many bags are required in each bag contained in a
shiny gold bag and adding them all up. The base case
is bags that don't contain any other bags.

In [6]:
def count_bags(color: str):
    bag_count = 0
    for inner_bag in color_contains[color]:
        num = inner_bag.num
        inner_color = inner_bag.color
        bag_count += num * (1 + count_bags(inner_color))

    return bag_count

count_bags("shiny gold")

6683